In [4]:
import selenium
from selenium import webdriver
from selenium.webdriver import Chrome as wdChrome, Firefox as wdFirefox, Edge as wdEdge
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
# from selenium.webdriver.edge.service import Service as EdgeService
# from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions as selExcep 

from webdriver_manager.chrome import ChromeDriverManager
# from webdriver_manager.firefox import GeckoDriverManager
# from webdriver_manager.microsoft import EdgeChromiumDriverManager


import pandas as pd
import numpy as np
import time

In [5]:
home_url = 'https://www.google.com/maps/place/Lahore,+Punjab,+Pakistan/@74.1943055,31.4831569,11z/data=!3m1!4b1!4m5!3m4!1s0x39190483e58107d9:0xc23abe6ccc7e2462!8m2!3d31.5203696!4d74.3587473'
driver_type = "chrome"
service = ChromeService(executable_path = ChromeDriverManager().install())
driver = wdChrome(service= service)
driver.get(home_url)
action = ActionChains(driver)
time.sleep(1)

[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:32<00:00, 217kB/s]


In [6]:
MAP_SEARCH_XPATH = '//*[@id="searchboxinput"]'
LOCATION_BOX_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]'
LOC_BOX_CLASS = "hfpxzc"

In [7]:
LOC_NAME_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1'
LOC_RATING_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[1]/span/span[1]'
LOC_RATING_COUNT_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/button'

In [8]:
def search_map(query: str) -> None:
    map_searchbar = driver.find_element(By.XPATH, MAP_SEARCH_XPATH)
    map_searchbar.clear()
    map_searchbar.send_keys(query)
    map_searchbar.send_keys(Keys.RETURN)

### Collecting hrefs from all locations in a radius

need to add functionality to search all radius in a lat-long data grid

In [9]:
## Covers all of lahore

start_lat= 31.283179
start_long= 74.080275

stop_lat = 31.671291
stop_long = 74.551132

lat_step = (stop_lat - start_lat) / 3
long_step = (stop_long - start_long) / 3

lat_dist = np.arange(start=start_lat, stop= stop_lat, step= lat_step, dtype= "float32")
long_dist = np.arange(start=start_long, stop= stop_long, step= long_step, dtype= "float32")

geo_loc_grid = np.transpose([np.tile(lat_dist, len(long_dist)), np.repeat(long_dist, len(lat_dist))])

In [10]:
count = 0
for geo_loc in geo_loc_grid:
    print(geo_loc)
    count += 1
count

[31.283178 74.08028 ]
[31.383179 74.08028 ]
[31.48318 74.08028]
[31.58318 74.08028]
[31.283178 74.180275]
[31.383179 74.180275]
[31.48318  74.180275]
[31.58318  74.180275]
[31.283178 74.28027 ]
[31.383179 74.28027 ]
[31.48318 74.28027]
[31.58318 74.28027]
[31.283178 74.38027 ]
[31.383179 74.38027 ]
[31.48318 74.38027]
[31.58318 74.38027]
[31.283178 74.48027 ]
[31.383179 74.48027 ]
[31.48318 74.48027]
[31.58318 74.48027]


20

In [11]:
print(lat_dist.size)
print(lat_dist)
print()

print(long_dist.size)
print(long_dist)
print()

print(lat_dist.size * long_dist.size)

4
[31.283178 31.383179 31.48318  31.58318 ]

5
[74.08028  74.180275 74.28027  74.38027  74.48027 ]

20


In [15]:
search_map("resteraunts")

In [13]:
# loc_div = driver.find_element(By.XPATH, LOCATION_BOX_XPATH)
# action.move_to_element(loc_div).perform()
# hrefs_set = set()


# last_href = None
# find_elem = False
# while not find_all_elem:
    
#     loc_boxes = driver.find_elements(By.CLASS_NAME, LOC_BOX_CLASS)
#     hrefs = [loc_box.get_attribute("href") for loc_box in loc_boxes]
#     hrefs_set.update(hrefs)
#     loc_div.send_keys(Keys.END)
#     time.sleep(2)
    
#     if(last_href == None):
#         last_href = hrefs[-1]
#         continue
    
#     if(last_href == hrefs[-1]):
#         find_all_elem = True
#     else:
#         last_href = hrefs[-1]

In [16]:
hrefs_set = set()


last_href = None
find_all_elem = False
curr_url = driver.current_url
template_url = curr_url.split("@")[0]

for loc in geo_loc_grid:
    lat = loc[0]
    long = loc[1]
    dynamic_url = f"{template_url}@{lat},{long},13z/data=!3m1!4b1"
    print(dynamic_url)
    driver.get(dynamic_url)
    time.sleep(1)
    while not find_all_elem:
        loc_div = driver.find_element(By.XPATH, LOCATION_BOX_XPATH)
        action.move_to_element(loc_div).perform()
        loc_boxes = driver.find_elements(By.CLASS_NAME, LOC_BOX_CLASS)
        hrefs = [loc_box.get_attribute("href") for loc_box in loc_boxes]
        hrefs_set.update(hrefs)
        loc_div.send_keys(Keys.END)
        time.sleep(2)
        
        if(last_href == None):
            last_href = hrefs[-1]
            continue
        
        if(last_href == hrefs[-1]):
            find_all_elem = True
        else:
            last_href = hrefs[-1]
    find_all_elem = False
    time.sleep(5)

https://www.google.com/maps/search/resteraunts/@31.283178329467773,74.08027648925781,13z/data=!3m1!4b1


In [17]:
hrefs_set

{'https://www.google.com/maps/place/%D8%B3%D8%B1%D9%88%D8%A7%D9%86%DB%8C+%D8%B1%D8%AD%D9%85%DA%A9%D8%A7%D8%B1%DB%8C%DB%81+%DB%81%D9%88%D9%B9%D9%84%E2%80%AD/data=!4m7!3m6!1s0x3918572a17d60c9d:0x2c9eb95fa06ba190!8m2!3d31.3299377!4d74.1096191!16s%2Fg%2F11m0v0_7p0!19sChIJnQzWFypXGDkRkKFroF-5niw?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/%D9%85%D8%B1%D8%AD%D8%A8%D8%A7+%D8%B3%D9%88%DB%8C%D9%B9%D8%B3+%D8%A7%DB%8C%D9%86%DA%88+%D8%A8%DB%8C%DA%A9%D8%B1%E2%80%AD/data=!4m7!3m6!1s0x39185ba1b85d00f9:0x79dd94d5fa6b2842!8m2!3d31.2467849!4d74.0055149!16s%2Fg%2F11m87_ny76!19sChIJ-QBduKFbGDkRQihr-tWU3Xk?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/114+by+Insomnia+Kitchen/data=!4m7!3m6!1s0x3919058cbd3e17db:0xb3fc86f146c33aa3!8m2!3d31.4888157!4d74.326597!16s%2Fg%2F11h5sqklr_!19sChIJ2xc-vYwFGTkRozrDRvGG_LM?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/AL+SHAHEEN+RESTAURANT/data=!4m7!3m6!1s0x39185b7ed45639cd:0x7c383cf927f1bb46!8m2!3d31.2501103!4d74.0111329!

# Extracting data from href

In [18]:
df = pd.DataFrame()

### getting data from each href

In [30]:
href_list = list()
loc_names = list()
loc_ratings = list()
latitudes = list()
longitudes = list()
total_ratings = list()


In [31]:
href_list = list(hrefs_set)
href_list.sort()

### Getting infro from each href

In [32]:
for href in href_list:
    lat, long = href.split("!3d")[1].split("!4d")
    long = long.split("!")[0]
    
    driver.get(href)
    time.sleep(0.5)
    
    name = driver.find_element(By.XPATH, LOC_NAME_XPATH).text
    
    try:
        rating = driver.find_element(By.XPATH, LOC_RATING_XPATH).text
        rating_count = driver.find_element(By.XPATH, LOC_RATING_COUNT_XPATH).text.split()[0]
        rating_count = rating_count.replace(",", "")
        rating_count = int(rating_count)  
    except selExcep.NoSuchElementException:
        rating = None
        rating_count = None
          
    loc_names.append(name)
    loc_ratings.append(rating)
    latitudes.append(lat)
    longitudes.append(long)
    total_ratings.append(rating_count)

In [35]:
df["loc_name"] = loc_names
df["loc_rating"] = loc_ratings
df["total_rating"] = total_ratings
df["latitude"] = latitudes
df["longitude"] = longitudes
df

,loc_name,loc_rating,total_rating,latitude,longitude
0,سروانی رحمکاریہ ہوٹل,3.4,10.0,31.3299377,74.1096191
1,مرحبا سویٹس اینڈ بیکر,5.0,2.0,31.2467849,74.0055149
2,114 by Insomnia Kitchen,4.1,131.0,31.4888157,74.326597
3,AL SHAHEEN RESTAURANT,4.2,52.0,31.2501103,74.0111329
4,AL-Ahmed Hotel & Restaurant,5.0,1.0,31.3364077,74.1151613
5,Al Kareem Restaurant,4.6,14.0,31.4649812,74.3145004
6,Al Khaliq Restaurant AKR,5.0,3.0,31.3028831,74.1727406
7,Al Khan Restaurant,4.3,2057.0,31.2783164,74.0548248
8,Anwar-e-Sidra Family Restaurant,3.8,36.0,31.2398772,73.9953167
9,Ayub Sweets & Samosa Shop,5.0,1.0,31.3027272,74.077951


In [36]:
df.to_csv('prototype_2.csv')

In [22]:
test_href = hrefs_set.pop()
test_href

'https://www.google.com/maps/place/Qabail/data=!4m7!3m6!1s0x39190451441bfde9:0x90460475ef116281!8m2!3d31.5148003!4d74.3514252!16s%2Fg%2F11cjkmjk2h!19sChIJ6f0bRFEEGTkRgWIR73UERpA?authuser=0&hl=en&rclk=1'

In [23]:
driver.get(test_href)

In [25]:
rating_count = driver.find_element(By.XPATH, LOC_RATING_COUNT_XPATH).text.split()[0]
rating_count

'2,780'